In [1]:
import os
import glob
import pandas as pd
import numpy as np
import random
from collections import Counter
from vit_keras import vit, utils
import tensorflow_addons as tfa
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.metrics

In [2]:
IMAGE_SIZE = "CHANGE"

working_dir = '/Users/suzyxu/Documents/ML/Mohs2023/'
raw_data = working_dir + "data/RAW/"
processed_data = working_dir + "data/processed/"
model_folder = working_dir + "data/model/"

MODEL_PARAMETERS = {"model_name" : "Mohs_BCC_SCC_2023", #name of the model of my choosing                    
                    "loss" : "categorical_crossentropy", #
                    "class_mode": "categorical", #
                    "output_layers" : 3, #
                    "output_activation" : "softmax", #
                    "activation" : "relu", 
                    "optimizer" : "sgd",
                    "num_epochs" : 50,
                    "batch_size" : 128}

model_output_folder = model_folder + MODEL_PARAMETERS["model_name"] + "/"

In [3]:
if not os.path.isdir(model_output_folder):
    os.mkdir(model_output_folder)

# Write out parameters
with open(model_output_folder+'MODEL_PARAMETERS.txt','w') as data: 
      data.write(str(MODEL_PARAMETERS))
        
print(MODEL_PARAMETERS)

{'model_name': 'Mohs_BCC_SCC_2023', 'loss': 'categorical_crossentropy', 'class_mode': 'categorical', 'output_layers': 3, 'output_activation': 'softmax', 'activation': 'relu', 'optimizer': 'sgd', 'num_epochs': 50, 'batch_size': 128}


In [4]:
#putting all photos in one table with full path
image_pattern = raw_data + "/**/*.jpg"
file_list = glob.glob(image_pattern, recursive=True)
Training_table = pd.DataFrame(file_list,columns = ["full_path"])
Training_table

,full_path
0,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
1,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
2,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
3,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
4,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
...,...
291,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
292,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
293,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...
294,/Users/suzyxu/Documents/ML/Mohs2023/data/RAW/P...


In [8]:
Training_table["metadata"] = [file.split("/")[10] for file in Training_table["full_path"]]
Training_table["metadata"] = [file.replace(".jpg", "") for file in Training_table["metadata"]]
Training_table = Training_table[ Training_table[ "metadata" ].str.contains( "2x" )==False ]
Training_table.reset_index(drop = True,inplace = True)

Training_table["patient"] = ""
Training_table["slide"] = ""
Training_table["zoom"] = ""
Training_table["cancer"] = ""
Training_table["image_number"] = ""
Training_table["class_set"] = "train"

for index in Training_table.index:
    metadata = Training_table["metadata"][index]
    metadata = metadata.split("_")
    Training_table.loc[index,"patient"] = metadata[0]
    Training_table.loc[index,"slide"] = metadata[1]
    Training_table.loc[index,"zoom"] = metadata[2]
    Training_table.loc[index,"cancer"] = metadata[3]
    Training_table.loc[index,"image_number"] = metadata[4]
    
random.seed(10)
test_sample = Training_table.groupby(["patient","cancer"], group_keys=False).apply(lambda x:x.sample(frac=0.3)).index.tolist()
Training_table.loc[Training_table.index[test_sample],"class_set"] = "test"
Training_table.groupby(["patient", "cancer", "class_set"]).size()

random.seed(10)
val_sample = Training_table.loc[test_sample].groupby(["patient","cancer"], group_keys=False).apply(lambda x:x.sample(frac=0.3)).index.tolist()
Training_table.loc[Training_table.index[val_sample],"class_set"] = "val"

Training_table.groupby(["patient", "cancer", "class_set"]).size()

patient  cancer  class_set
1        BCC     test          1
                 train         3
                 val           1
         N       test          6
                 train        20
                 val           3
2        BCC     test          8
                 train        26
                 val           3
         N       test          1
                 train         3
3        N       test         11
                 train        36
                 val           5
         SCC     test         10
                 train        31
                 val           4
4        BCC     test          8
                 train        28
                 val           4
         N       test          7
                 train        23
                 val           3
dtype: int64